In [66]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler
import os
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import copy

In [3]:
for root, dirs, files in os.walk("raw data 17-19"):
    raw_list=files

In [19]:
features = [
    'fastDlag', 'MA5lag', 'day5Returnlag', 'PROClag', 'day3Returnlag',
    'MA3lag', 'Closelag', 'Close','Up'
]

In [5]:
os.path.join('raw data 17-19',raw_list[1])

'raw data 17-19\\SH600000.csv'

In [102]:
tt_list=[]
oot_list=[]
backup_tt=[]
backup_oot=[]
for i in raw_list[27:39]: 
    data=pd.read_csv(os.path.join('raw data 17-19',i))
    variables=pd.read_csv(os.path.join('engineered features 17-19',i))
    variables.rename(columns={'Unnamed: 0':'Time'},inplace=True)
    data=data.merge(variables,on='Time',how='left')
    #MACD
    data['EMA-12']=np.array(0)
    N=12.0
    coeff=2.0/(N+1)
    data.loc[0,'EMA-12'] = data.loc[0,'Close']
    for i in range(1,len(data)):
        data.loc[i,'EMA-12'] = (data.loc[i-1,'EMA-12']*(1-coeff))+ (data.loc[i,'Close']*coeff)
    data['EMA-26']=np.array(0)
    ## 26day EMA
    N=26.0
    coeff=2.0/(N+1)
    data.loc[0,'EMA-26'] = data.loc[0,'Close']
    for i in range(1,len(data)):
        data.loc[i,'EMA-26'] = (data.loc[i-1,'EMA-26']*(1-coeff))+ (data.loc[i,'Close']*coeff)
    data['DIF'] = data['EMA-12']-data['EMA-26']
    N=9
    coeff=2.0/(N+1)
    data['MACD']=np.array(0)
    data.loc[0,'MACD'] = 0
    for i in range(2,len(data)):
        data.loc[i,'MACD']= (data.loc[i-1,'MACD']*(1-coeff))+ (data.loc[i,'DIF']*coeff)
    data.drop(['EMA-12','EMA-26','DIF'],axis=1,inplace=True)
    #3-Day Return & MA3
    ## 3-Day Return
    price = data["Close"]
    day3Return = np.zeros(len(data))
    for i in range(14,len(data)):
        day3Return[i] = 100*(price[i]-price[i-2])/price[i-2] 
    day3Returnlag = np.append(np.array([0]),day3Return[0:len(day3Return)-1])
    data['day3Returnlag']=day3Returnlag
    ## MA3
    MA3 = np.zeros(len(data))
    for i in range(14,len(data)):
        MA3[i] = data.loc[i-2:i+1,'Close'].sum()/39
    MA3lag = np.append(np.array([0]),MA3[0:len(MA3)-1])
    data['MA3lag']=MA3lag
    Openlag = np.append(np.array([0]),data['Open'][0:len(data)-1])
    data['Openlag']=Openlag
    Highlag = np.append(np.array([0]),data['High'][0:len(data)-1])
    data['Highlag']=Highlag
    Lowlag = np.append(np.array([0]),data['Low'][0:len(data)-1])
    data['Lowlag']=Lowlag
    Closelag = np.append(np.array([0]),data['Close'][0:len(data)-1])
    data['Closelag']=Closelag
    data['Up']=data['Close']>data['Closelag']
    data.drop(['Open','High','Low','Volume'],axis=1,inplace=True)
    data=copy.deepcopy(data.iloc[16:,:])
    tt=copy.deepcopy(data.iloc[:404,:])
    oot=copy.deepcopy(data.iloc[404:504,:])
     ##Binary 
    tt_time=tt['Time'].values
    oot_time=oot['Time'].values
    tt_y=tt['Close'].values
    tt_up=tt['Up'].values
    oot_y=oot['Close'].values
    oot_up=oot['Up'].values
    tt.drop(['Time','Close','Up'],inplace=True,axis=1)
    oot.drop(['Time','Close','Up'],inplace=True,axis=1)
    columns=tt.columns
    ss = StandardScaler()
    tt= ss.fit_transform(tt)
    oot = ss.transform(oot)
    tt=pd.DataFrame(tt,columns=columns)
    oot=pd.DataFrame(oot,columns=columns)
    tt['Time']=tt_time
    oot['Time']=oot_time
    tt['Close']=tt_y
    oot['Close']=oot_y
    tt['Up']=tt_up
    oot['Up']=oot_up
    tt=tt[features].copy()
    oot=oot[features].copy()
    tt_list.append(tt)
    oot_list.append(oot)

In [106]:
data

,Time,Close,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,...,day15Returnlag,PROClag,MACD,day3Returnlag,MA3lag,Openlag,Highlag,Lowlag,Closelag,Up
16,Day17,6.9616,74006700.0,63.701119,0.583289,0.656972,-0.500000,1.163648e+09,7.14578,7.090627,...,2.927681,0.218932,0.040222,-1.295161,0.729218,7.1177,7.1801,7.1177,7.1333,False
17,Day18,6.9148,125354600.0,49.400734,0.086908,0.403947,-0.692459,1.038294e+09,7.12080,7.092707,...,-0.888383,-2.436463,0.035901,-2.193124,0.721215,7.1177,7.1333,6.9304,6.9616,False
18,Day19,6.9304,106382000.0,46.346661,0.076885,0.249027,-0.200000,9.319116e+08,7.07086,7.085427,...,-2.637248,-0.674529,0.030155,-3.063098,0.716413,6.9460,6.9616,6.8836,6.9148,True
19,Day20,6.8836,49766300.0,47.511520,0.115328,0.093040,0.500000,9.816779e+08,7.01156,7.073980,...,-2.844406,0.225349,0.023002,-0.448173,0.710010,6.9148,6.9460,6.8836,6.9304,False
20,Day21,6.8992,102928200.0,44.397368,0.133461,0.108558,0.000801,8.787497e+08,6.96474,7.057333,...,-2.432249,-0.677576,0.015539,-0.451206,0.708410,6.9304,6.9460,6.8211,6.8836,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Day752,10.5852,166841712.0,62.296944,0.923160,0.831739,0.500351,1.419082e+10,10.42128,10.316713,...,2.418128,1.016753,0.007373,1.541270,1.077438,10.4605,10.6030,10.4605,10.5674,True
752,Day753,10.4783,120161784.0,63.007456,0.961580,0.889123,0.666978,1.431099e+10,10.46404,10.334533,...,2.237891,0.168301,0.014756,1.192104,1.079267,10.5496,10.6030,10.4961,10.5852,False
753,Day754,10.4426,119138984.0,56.162314,0.730844,0.871861,-0.427885,1.419185e+10,10.49968,10.342853,...,1.554580,-1.015035,0.020490,-0.843159,1.078808,10.5674,10.5674,10.4426,10.4783,False
754,Day755,10.3892,68218760.0,54.050454,0.653788,0.782071,-0.001403,1.412363e+10,10.50680,10.351167,...,1.736098,-0.341286,0.023992,-1.347164,1.074238,10.4426,10.4783,10.4070,10.4426,False


In [86]:
oot_list

IndexError: list index out of range

In [28]:
for i in range(len(tt_list)):
    #Random Forest
    X = tt_list[i][[
        'fastDlag', 'MA5lag', 'day5Returnlag', 'PROClag', 'day3Returnlag',
        'MA3lag', 'Closelag'
    ]]
    y = tt_list[i]['Up']
    rf = RandomForestClassifier()
    param_grid = {
        'n_estimators': [100, 200, 400, 800,1000]
    }
    grid = GridSearchCV(rf, param_grid=param_grid, scoring='accuracy', cv=5)
    grid.fit(X, y)
    print('Stock:', i)
    print('Best Param:', grid.best_params_)
    print('Best Score:', grid.best_score_)

Stock: 0
Best Param: {'n_estimators': 800}
Best Score: 0.551980198019802
Stock: 1
Best Param: {'n_estimators': 400}
Best Score: 0.4405940594059406
Stock: 2
Best Param: {'n_estimators': 400}
Best Score: 0.5024752475247525
Stock: 3
Best Param: {'n_estimators': 100}
Best Score: 0.4801980198019802
Stock: 4
Best Param: {'n_estimators': 100}
Best Score: 0.556930693069307
Stock: 5
Best Param: {'n_estimators': 400}
Best Score: 0.5495049504950495
Stock: 6
Best Param: {'n_estimators': 100}
Best Score: 0.5445544554455446
Stock: 7
Best Param: {'n_estimators': 800}
Best Score: 0.5123762376237624
Stock: 8
Best Param: {'n_estimators': 400}
Best Score: 0.46534653465346537
Stock: 9
Best Param: {'n_estimators': 400}
Best Score: 0.5420792079207921
Stock: 10
Best Param: {'n_estimators': 200}
Best Score: 0.556930693069307
Stock: 11
Best Param: {'n_estimators': 800}
Best Score: 0.5024752475247525


In [27]:
for i in range(len(tt_list)):
    #SVM
    X=tt_list[i][[
    'fastDlag', 'MA5lag', 'day5Returnlag', 'PROClag', 'day3Returnlag',
    'MA3lag', 'Closelag']]
    y=tt_list[i]['Up']
    svm=SVC()
    param_grid= {'C': [1000,10000,100000,1000000,10000000], 'gamma': [0.1,0.01,0.001, 0.0001]}
    grid=GridSearchCV(svm,param_grid=param_grid,scoring='accuracy',cv=5)
    grid.fit(X,y)
    print('Stock:',i)
    print('Best Param:',grid.best_params_)
    print('Best Score:',grid.best_score_)

Stock: 0
Best Param: {'C': 10000000, 'gamma': 0.0001}
Best Score: 0.9306930693069307
Stock: 1
Best Param: {'C': 10000000, 'gamma': 0.0001}
Best Score: 0.9529702970297029
Stock: 2
Best Param: {'C': 10000000, 'gamma': 0.0001}
Best Score: 0.9133663366336634
Stock: 3
Best Param: {'C': 1000000, 'gamma': 0.001}
Best Score: 0.9207920792079208
Stock: 4
Best Param: {'C': 1000000, 'gamma': 0.001}
Best Score: 0.9579207920792079
Stock: 5
Best Param: {'C': 100000, 'gamma': 0.001}
Best Score: 0.9207920792079208
Stock: 6
Best Param: {'C': 10000000, 'gamma': 0.001}
Best Score: 0.9381188118811881
Stock: 7
Best Param: {'C': 1000000, 'gamma': 0.001}
Best Score: 0.9579207920792079
Stock: 8
Best Param: {'C': 10000000, 'gamma': 0.001}
Best Score: 0.8811881188118812
Stock: 9
Best Param: {'C': 1000000, 'gamma': 0.001}
Best Score: 0.9826732673267327
Stock: 10
Best Param: {'C': 10000000, 'gamma': 0.001}
Best Score: 0.9628712871287128
Stock: 11
Best Param: {'C': 1000000, 'gamma': 0.001}
Best Score: 0.94059405940

In [29]:
for i in range(len(tt_list)):
    #Random Forest Regressor
    X = tt_list[i][[
        'fastDlag', 'MA5lag', 'day5Returnlag', 'PROClag', 'day3Returnlag',
        'MA3lag', 'Closelag'
    ]]
    y = tt_list[i]['Close']
    rf = RandomForestRegressor()
    param_grid = {
        'n_estimators': [100, 200, 400, 800,1000]
    }
    grid = GridSearchCV(rf, param_grid=param_grid, cv=5)
    grid.fit(X, y)
    print('Stock:', i)
    print('Best Param:', grid.best_params_)
    print('Best Score:', grid.best_score_)

Stock: 0
Best Param: {'n_estimators': 200}
Best Score: 0.190649724074534
Stock: 1
Best Param: {'n_estimators': 200}
Best Score: 0.5207470969394472
Stock: 2
Best Param: {'n_estimators': 100}
Best Score: -0.6344565547781476
Stock: 3
Best Param: {'n_estimators': 200}
Best Score: 0.5946249840285209
Stock: 4
Best Param: {'n_estimators': 1000}
Best Score: 0.7791118191386442
Stock: 5
Best Param: {'n_estimators': 800}
Best Score: 0.06425865258670531
Stock: 6
Best Param: {'n_estimators': 100}
Best Score: 0.6829420092429418
Stock: 7
Best Param: {'n_estimators': 800}
Best Score: 0.16170511874331298
Stock: 8
Best Param: {'n_estimators': 200}
Best Score: 0.37874036963616176
Stock: 9
Best Param: {'n_estimators': 400}
Best Score: 0.7832394754767125
Stock: 10
Best Param: {'n_estimators': 200}
Best Score: 0.8318920352145639
Stock: 11
Best Param: {'n_estimators': 100}
Best Score: 0.08087179577516168


In [30]:
from sklearn.metrics import confusion_matrix

In [107]:
for i in range(len(tt_list)):
    #SVM
    X=tt_list[i][[
    'fastDlag', 'MA5lag', 'day5Returnlag', 'PROClag', 'day3Returnlag',
    'MA3lag', 'Closelag']]
    y=tt_list[i]['Up']
    svm=SVC(C=1000000,gamma=0.001)
    svm.fit(X,y)
    X_oot=oot_list[i][[
    'fastDlag', 'MA5lag', 'day5Returnlag', 'PROClag', 'day3Returnlag',
    'MA3lag', 'Closelag']]
    y_oot=oot_list[i]['Up'] 
    y_pred=svm.predict(X_oot)
    print(confusion_matrix(y_oot,y_pred))

[[46  5]
 [ 7 42]]
[[51  0]
 [ 9 40]]
[[45  4]
 [ 2 49]]
[[48  4]
 [ 1 47]]
[[52  0]
 [ 1 47]]
[[52  2]
 [ 1 45]]
[[50  5]
 [ 0 45]]
[[51  1]
 [ 2 46]]
[[50  6]
 [ 1 43]]
[[46  1]
 [ 0 53]]
[[52  1]
 [ 3 44]]
[[56  0]
 [ 5 39]]


420    NaN
421    NaN
422    NaN
423    NaN
424    NaN
      ... 
515    NaN
516    NaN
517    NaN
518    NaN
519    NaN
Name: Up, Length: 100, dtype: object

In [47]:
oot_list[0]

,fastDlag,MA5lag,day5Returnlag,PROClag,day3Returnlag,MA3lag,Closelag,Close,Up
420,-6.565248,-2.630195,-6.975352,-5.866537,-6.312660,-6.232300,-2.588199,8.8718,True
421,-6.554818,-2.633124,-4.814694,-6.481031,-5.691373,-6.229421,-2.573554,9.0207,True
422,-6.514900,-2.588199,-5.366896,-5.882299,-5.707459,-6.226792,-2.505198,8.9144,False
423,-6.503643,-2.575501,-5.466351,-7.190588,-6.425961,-6.228544,-2.553998,8.6910,False
424,-6.521991,-2.575501,-7.419672,-7.811534,-8.324292,-6.228920,-2.656556,8.8399,True
...,...,...,...,...,...,...,...,...,...
515,-6.598507,-2.803075,-6.989282,-5.361521,-5.525344,-6.251335,-2.754247,8.5633,True
516,-6.570403,-2.801119,-5.296543,-6.187896,-4.869493,-6.245575,-2.715180,8.7654,True
517,-6.509686,-2.760105,-3.932238,-5.575530,-5.091268,-6.232051,-2.622400,9.3931,True
518,-6.405537,-2.657547,-0.248760,-3.471270,-2.197850,-6.223285,-2.334237,9.2229,False
